# Alerting & Automation

- Loads cleaned sessions, runs daily network-type aggregates.
- Applies threshold logic to flag days with bad QoE.
- Outputs flagged breaches to `network_qoe_alerts.csv`.
- Optional: sends alerts via Telegram if configured.
- Prints summary stats and shows recent alerts for review.


In [ ]:
# Step 1: Imports & load
import os
import pandas as pd
import numpy as np

if os.path.exists("synthetic_qoe_sessions_clean.parquet"):
    df = pd.read_parquet("synthetic_qoe_sessions_clean.parquet")
else:
    df = pd.read_csv("synthetic_qoe_sessions_clean.csv", parse_dates=['timestamp'])

print(f"Loaded cleaned dataset with shape: {df.shape}")
display(df.head())


In [ ]:
# Step 2: Thresholds & Telegram config (env-driven)
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env if present

def _getenv_required(key: str) -> str:
    v = os.getenv(key)
    if not v:
        raise RuntimeError(f"Missing required env var: {key}")
    return v

# Thresholds (env overrides; defaults retained)
PAGE_LOAD_THRESHOLD = 2500        # ms
BUFFERING_RATIO_THRESHOLD = 0.10  # ratio
STARTUP_DELAY_THRESHOLD = 1000    # ms

# Secrets (required)
TELEGRAM_BOT_TOKEN = _getenv_required("TELEGRAM_BOT_TOKEN")
TELEGRAM_CHAT_ID   = _getenv_required("TELEGRAM_CHAT_ID")

# Safer default for demos: dry-run unless explicitly disabled
DRY_RUN = False  # set to False to actually send to Telegram

print("[Alerting config]")
print(f"- Thresholds: PLT>{PAGE_LOAD_THRESHOLD}ms, BUF>{BUFFERING_RATIO_THRESHOLD}, SDL>{STARTUP_DELAY_THRESHOLD}ms")
print(f"- Telegram: chat={TELEGRAM_CHAT_ID}, dry_run={DRY_RUN}")


In [ ]:
# Step 3: Daily aggregation by network_type (plus median radio for 4G/5G)
df['date'] = df['timestamp'].dt.floor('D')
daily = (
    df.groupby(['date','network_type'], observed=True)
      .agg(page_load_time_ms=('page_load_time_ms','mean'),
           buffering_ratio=('buffering_ratio','mean'),
           startup_delay_ms=('startup_delay_ms','mean'),
           rsrp_dbm=('rsrp_dbm','median'),
           sinr_db=('sinr_db','median'),
           n=('session_id','count'))
      .reset_index()
)
display(daily.head())


In [ ]:
# Step 4: Build alert rows
latest_date = daily['date'].max()
d = daily[daily['date'] == latest_date].copy()

alerts = []
for _, r in d.iterrows():
    breaches = []
    if r['page_load_time_ms'] > PAGE_LOAD_THRESHOLD:
        breaches.append(f"PLT>{PAGE_LOAD_THRESHOLD}ms ({int(r['page_load_time_ms'])}ms)")
    if r['buffering_ratio'] > BUFFERING_RATIO_THRESHOLD:
        breaches.append(f"BUF>{BUFFERING_RATIO_THRESHOLD:.2f} ({r['buffering_ratio']:.3f})")
    if r['startup_delay_ms'] > STARTUP_DELAY_THRESHOLD:
        breaches.append(f"SDL>{STARTUP_DELAY_THRESHOLD}ms ({int(r['startup_delay_ms'])}ms)")

    if breaches:
        radio_flag = ""
        if r['network_type'] in ['4G','5G']:
            bad_radio = (pd.notna(r['rsrp_dbm']) and r['rsrp_dbm'] < -110) or (pd.notna(r['sinr_db']) and r['sinr_db'] < 0)
            radio_flag = " ⚠️ radio" if bad_radio else ""
        alerts.append({
            "date": latest_date.strftime("%Y-%m-%d"),
            "network_type": r['network_type'],
            "breaches": ", ".join(breaches),
            "n_sessions": int(r['n']),
            "rsrp_dbm_med": None if pd.isna(r['rsrp_dbm']) else float(r['rsrp_dbm']),
            "sinr_db_med": None if pd.isna(r['sinr_db']) else float(r['sinr_db']),
            "radio_flag": radio_flag
        })

alerts_df = pd.DataFrame(alerts)
print(f"Alerts for {latest_date.date()}: {len(alerts_df)}")
display(alerts_df.head())


In [ ]:
# Step 5: Telegram sender (single concise message for latest date)
def send_telegram_message(token: str, chat_id: str, text: str) -> tuple[bool, str]:
    try:
        import requests
    except ModuleNotFoundError:
        return False, "requests not installed. Try: conda install -n qoe-demo requests"

    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {"chat_id": chat_id, "text": text}
    try:
        resp = requests.post(url, data=payload, timeout=10)
        if resp.status_code == 200 and resp.json().get("ok"):
            return True, "sent"
        return False, f"HTTP {resp.status_code}: {resp.text[:200]}"
    except Exception as e:
        return False, str(e)

if alerts_df.empty:
    print("No KPI breaches today — no Telegram message sent.")
else:
    # Build compact message
    header = f"📡 QoE Alerts — {latest_date.strftime('%Y-%m-%d')}"
    lines = []
    for _, a in alerts_df.sort_values("network_type").iterrows():
        lines.append(f"- {a['network_type']}: {a['breaches']} (n={a['n_sessions']}){a['radio_flag']}")
    msg = header + "\n" + "\n".join(lines)

    if DRY_RUN:
        print("DRY_RUN=True — preview message:")
        print(msg)
        sent, info = True, "preview only"
    else:
        sent, info = send_telegram_message(TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, msg)

    print(f"Telegram send status: {sent} ({info})")


In [ ]:
# Step 6: Export alert summary
alerts_df.to_csv("network_qoe_alerts.csv", index=False)
print("Alert summary exported to network_qoe_alerts.csv")
